In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import KFold
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import networkx as nx

In [2]:
root_dir = "./archive/"
trainDF = []
testDF = []

features = ['DOY', 'DOW', 'CRS_DEP_TIME', 'DISTANCE']
target = ['ARR_DELAY']

for (root, dirs, filenames) in os.walk("./archive/airline delay analysis"):
    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(root, filename)
            
            name, extension = os.path.splitext(filename)
            
            df = pd.read_csv(file_path)

            df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], format='%Y-%m-%d')
            df['DOY']=df['FL_DATE'].dt.dayofyear
            df['DOW']=df['FL_DATE'].dt.dayofweek
            #df['YEAR'] = df['FL_DATE'].dt.year
            #df['MONTH'] = df['FL_DATE'].dt.month
            #df['DAY'] = df['FL_DATE'].dt.day

            df = df[features + target]
            if name == '2018':
                testDF = df
            elif name == '2009': 
                trainDF.append(df)

trainDF = list(trainDF)
trainDF = pd.concat(trainDF, ignore_index=True)

columnToCheck = 'ARR_DELAY'
condition = trainDF[columnToCheck].abs() > 60
trainDF = trainDF[~condition]
condition = testDF[columnToCheck].abs() > 60
testDF = testDF[~condition]

trainDF = trainDF.dropna()
testDF = testDF.dropna()

train_x = trainDF[features]
train_y = trainDF[target]

test_x = testDF[features]
test_y = testDF[target]

In [9]:
from torch.utils.data import TensorDataset
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

class SimpleRegressor(nn.Module):
    def __init__(self):
        super(SimpleRegressor, self).__init__()
        self.fc1 = nn.Linear(4, 32)
        self.fc2 = nn.Linear(16,64)
        self.fc3 = nn.Linear(32, 8)
        self.fc4 = nn.Linear(8, 1)
        #self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        #x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        #x = torch.clamp(x, -66, 66)
        #x = self.dropout(x)
        return x

def train_model(model, train_loader, criterion, optimizer, epochs):
    #clip_value = 1.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.view(-1)
            labels = labels.view(-1)
            #print(outputs, labels)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        #scheduler.step()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")

def accuracy(predictions, targets, threshold=6):
    absolute_errors = torch.abs(predictions - targets)
    correct_predictions = (absolute_errors <= threshold).float()
    accuracy = correct_predictions.mean().item()
    return accuracy

def evaluate_model(model, test_features, test_labels, criterion, threshold=6):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    total_accuracy = 0.0

    with torch.no_grad():
        outputs = model(test_features)
        print(outputs)
        labels = test_labels.view(-1)
        loss = criterion(outputs.squeeze(), labels)
        total_loss += loss.item()
        
        outputs = outputs.view(-1)
        acc = accuracy(outputs, labels, threshold=threshold)
        total_accuracy += acc

        total_samples += labels.size(0)

    mean_loss = total_loss
    mean_accuracy = total_accuracy
    print(f"Loss: {mean_loss}, Accuracy: {mean_accuracy}")

    return mean_loss, mean_accuracy

In [ ]:
# PyTorch 텐서로 변환
train_x_tensor = torch.tensor(train_x.values, dtype=torch.float32)
train_y_tensor = torch.tensor(train_y.values, dtype=torch.float32)

# PyTorch 텐서로 변환
test_x_tensor = torch.tensor(test_x.values, dtype=torch.float32)
test_y_tensor = torch.tensor(test_y.values, dtype=torch.float32)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_x_tensor = train_x_tensor.to(device)
train_y_tensor = train_y_tensor.to(device)

test_x_tensor = test_x_tensor.to(device)
test_y_tensor = test_y_tensor.to(device)

model = SimpleRegressor()
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_losses = []
fold_accuracies = []

for fold, (train_index, test_index) in enumerate(kf.split(train_x_tensor)):
    print(f"Fold {fold + 1}")
    
    fold_train_x, fold_test_x = train_x_tensor[train_index], train_x_tensor[test_index]
    fold_train_y, fold_test_y = train_y_tensor[train_index], train_y_tensor[test_index]

    # Move fold data to PyTorch tensors and then to the device
    fold_train_x = fold_train_x.to(device)
    fold_train_y = fold_train_y.to(device)
    fold_test_x = fold_test_x.to(device)
    fold_test_y = fold_test_y.to(device)

    # Create DataLoader for training and testing data
    train_dataset = TensorDataset(fold_train_x, fold_train_y)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(fold_test_x, fold_test_y)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    # Create an instance of the AirlineDelayPredictionModel class
    model = SimpleRegressor().to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    #scheduler = StepLR(optimizer, step_size=1, gamma=0.2)
    # Train the model
    train_model(model, train_loader, criterion, optimizer, epochs=1)
    
    # Evaluate the model on the test data
    fold_loss, fold_accuracy = evaluate_model(model, fold_test_x, fold_test_y, criterion, threshold=6)
    print(f"Fold {fold + 1} - Test Loss: {fold_loss}, Accuracy: {fold_accuracy}")

    fold_losses.append(fold_loss)
    fold_accuracies.append(fold_accuracy)

average_loss = sum(fold_losses) / len(fold_losses)
average_accuracy = sum(fold_accuracies) / len(fold_accuracies)

print(f"Overall Performance - Average Test Loss: {average_loss}, Average Accuracy: {average_accuracy}")

Fold 1
Epoch 1/1, Loss: 285.1103866792119
tensor([[-1.2188],
        [-1.2188],
        [-1.2188],
        ...,
        [-1.2188],
        [-1.2188],
        [-1.2188]], device='cuda:0')
Loss: 283.6912536621094, Accuracy: 0.328779399394989
Fold 1 - Test Loss: 283.6912536621094, Accuracy: 0.328779399394989
Fold 2


In [8]:
loss, acciracy = evaluate_model(model, test_x_tensor, test_y_tensor, criterion, threshold=6)
print(f'Loss: {loss}, accuracy: {accuracy}')

tensor([[-1.0814],
        [-1.0814],
        [-1.0814],
        ...,
        [-1.0814],
        [-1.0814],
        [-1.0814]], device='cuda:0')
correct_predictions : tensor([0., 0., 0.,  ..., 0., 0., 1.], device='cuda:0')
0.2766715884208679
Loss: 323.1086730957031, Accuracy: 0.2766715884208679
Loss: 323.1086730957031, accuracy: <function accuracy at 0x00000145D12FCCA0>


In [ ]:
# 훈련
num_epochs = 50
for epoch in range(num_epochs):
    # Forward pass
    
    outputs = model(train_x_tensor)
    loss = criterion(outputs, train_y_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation loss
    val_outputs = model(test_x_tensor)
    val_loss = criterion(val_outputs, test_y_tensor)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

num_epochs = 100
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(num_epochs):
    # Forward pass
    
    outputs = model(train_x_tensor)
    loss = criterion(outputs, train_y_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation loss
    val_outputs = model(test_x_tensor)
    val_loss = criterion(val_outputs, test_y_tensor)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

# 학습된 모델로 예측

loss, acciracy = evaluate_model(model, test_x_tensor, test_y_tensor, criterion, threshold=6)
print(f'Loss: {loss}, accuracy: {accuracy}')

In [ ]:
tDF['DOY_DOW_pair'] = list(zip(tDF['DOY'], tDF['DOW']))
print(tDF)

G = nx.Graph()

for idx, row in tDF.iterrows():
    G.add_node(idx, features=row[['DOY_DOW_pair', 'CRS_DEP_TIME', 'DISTANCE']].values, label=row['ARR_DELAY'])

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_losses = []
fold_accuracies = []

for fold, (train_index, test_index) in enumerate(kf.split(train_x_tensor)):
    print(f"Fold {fold + 1}")
    
    fold_train_x, fold_test_x = train_x_tensor[train_index], train_x_tensor[test_index]
    fold_train_y, fold_test_y = train_y_tensor[train_index] / 60, train_y_tensor[test_index] / 60

    # Move fold data to PyTorch tensors and then to the device
    fold_train_x = fold_train_x.to(device)
    fold_train_y = fold_train_y.to(device)
    fold_test_x = fold_test_x.to(device)
    fold_test_y = fold_test_y.to(device)

    # Create DataLoader for training and testing data
    train_dataset = TensorDataset(fold_train_x, fold_train_y)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(fold_test_x, fold_test_y)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    # Create an instance of the AirlineDelayPredictionModel class
    model = SimpleRegressor().to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.2)
    # Train the model
    train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20)
    
    # Evaluate the model on the test data
    fold_loss, fold_accuracy = evaluate_regression_model(model, test_loader, criterion, threshold=0.1)
    print(f"Fold {fold + 1} - Test Loss: {fold_loss}, Accuracy: {fold_accuracy}")

    fold_losses.append(fold_loss)
    fold_accuracies.append(fold_accuracy)

average_loss = sum(fold_losses) / len(fold_losses)
average_accuracy = sum(fold_accuracies) / len(fold_accuracies)

print(f"Overall Performance - Average Test Loss: {average_loss}, Average Accuracy: {average_accuracy}")

In [10]:
def select_subgraph_nodes(graph, num_nodes):
    return list(graph.nodes)[:num_nodes]

def train_subgraph_model(model, subgraph, features, labels, criterion, optimizer, num_epochs=100):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(features, subgraph)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
#def evaluate_subgraph_model(model, features, adjacency_matrix, labels, criterion):
#    with torch.no_grad():
#        output = model(features, adjacency_matrix)
#        loss = criterion(output, labels)
#        print(f'Subgraph Model Loss: {loss.item()}')

def accuracy(predictions, targets, threshold=0.1):
    absolute_errors = torch.abs(predictions - targets)
    correct_predictions = (absolute_errors <= threshold).float()
    accuracy = correct_predictions.mean().item()
    return accuracy

def evaluate_subgraph_model(model, test_features, test_adjacency, test_labels, criterion, threshold=0.1):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    total_accuracy = 0.0

    with torch.no_grad():
        outputs = model(test_features, test_adjacency)
        labels = test_labels.view(-1)
        loss = criterion(outputs.squeeze(), labels)
        total_loss += loss.item()

        # 정확도 계산
        acc = accuracy(outputs, labels, threshold=threshold)
        total_accuracy += acc

        total_samples += labels.size(0)

    mean_loss = total_loss / total_samples
    mean_accuracy = total_accuracy / total_samples
    print(f"Subgraph Model Loss: {mean_loss}, Accuracy: {mean_accuracy}")

    return mean_loss, mean_accuracy

In [11]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc1 = nn.Linear(input_dim, hidden_dim)
        self.gc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, adjacency_matrix):
        x = torch.matmul(x, self.gc1.weight.t())
        x = torch.matmul(adjacency_matrix, x.t()).t()
        x = torch.relu(x)
        x = torch.matmul(adjacency_matrix, x.t()).t()
        x = self.gc2(x)
        return x

In [12]:
import torch.optim as optim

input_dim = 3  # features의 차원
hidden_dim = 64
output_dim = 1  # 예측할 값이 하나이므로 output_dim을 1로 설정
subgraph_model = GCN(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
subgraph_optimizer = optim.Adam(subgraph_model.parameters(), lr=0.01)

# Subgraph 학습
num_subgraph_epochs = 10
num_nodes_per_subgraph = 64
#undirected_G = G.to_undirected()
#connected_components = list(nx.connected_components(undirected_G))
connected_components = list(nx.connected_components(G))

NetworkXNotImplemented: not implemented for directed type

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_nodes, test_nodes = train_test_split(list(G.nodes), test_size=0.2, random_state=42)

train_subgraph = G.subgraph(train_nodes)
test_subgraph = G.subgraph(test_nodes)

# train_subgraph의 노드를 직접 리스트로 변환하지 않고 순회하면서 데이터 추출
train_features = torch.FloatTensor([G.nodes[node]['features'] for node in train_subgraph.nodes()])
train_labels = torch.FloatTensor([G.nodes[node]['label'] for node in train_subgraph.nodes()])

# train_subgraph의 인접 행렬을 생성
train_adjacency = torch.Tensor(nx.linalg.graphmatrix.adjacency_matrix(train_subgraph).todense())

for subgraph_nodes in connected_components:
    # Subgraph에 사용할 일부 노드 선택
    subgraph_nodes = select_subgraph_nodes(undirected_G.subgraph(train_subgraph_nodes), num_nodes_per_subgraph)
    
    # Subgraph에 대한 학습 데이터 생성
    subgraph_features = train_features[subgraph_nodes]
    subgraph_labels = train_labels[subgraph_nodes]
    subgraph_adjacency = train_adjacency[subgraph_nodes][:, subgraph_nodes]
    
    # Subgraph 모델 학습
    train_subgraph_model(subgraph_model, subgraph_adjacency, subgraph_features, subgraph_labels, criterion, subgraph_optimizer, num_epochs=num_subgraph_epochs)
    
evaluate_subgraph_model(subgraph_model, test_features, test_adjacency, test_labels, criterion)

In [ ]:
connected_components = list(nx.connected_components(G))

for subgraph_nodes in connected_components:
    subgraph = G.subgraph(subgraph_nodes)
    
    subgraph_features = features[subgraph_nodes]
    subgraph_labels = labels[subgraph_nodes]
    
    

In [ ]:
adjacency_matrix = nx.linalg.graphmatrix.adjacency_matrix(subgraph).todense()
    
feature_matrix = np.array([row['DOY_DOW_pair', 'CRS_DEP_TIME', 'DISTANCE'] for idx, row in tDF.iterrows()])

label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(tDF['ARR_DELAY'])

In [ ]:
G = nx.DiGraph()

for idx, row in trainDF.iterrows():
    G.add_node(idx, features=row[['DOY', 'DOW', 'CRS_DEP_TIME', 'DISTANCE']].values, label=row['ARR_DELAY'])

In [ ]:
adjacency_matrix = nx.linalg.graphmatrix.adjacency_matrix(G).todense()
    
feature_matrix = np.array([row['DOY', 'DOW', 'CRS_DEP_TIME', 'DISTANCE'] for idx, row in trainDF.iterrows()])

label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(trainDF['ARR_DELAY'])

In [ ]:
class AirlineDelayPredictionModel(nn.Module):
    def __init__(self, timesteps, features):
        super(AirlineDelayPredictionModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=features, out_channels=9, kernel_size=1, stride=1)
        self.conv2 = nn.Conv1d(in_channels=9, out_channels=15, kernel_size=1, stride=1)
        self.conv3 = nn.Conv1d(in_channels=15, out_channels=24, kernel_size=1, stride=1)
        self.pool = nn.AvgPool1d(kernel_size=2, stride=1)
        self.lstm = nn.LSTM(input_size=24, hidden_size=200, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(400, 1)  # Adjusted input size of the linear layer to match LSTM output
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        padding = (0, 1)
        x = F.pad(x, padding)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.pad(x, padding)
        x = self.pool(x)
        x = self.conv3(x)
        x = F.pad(x, padding)
        x = self.pool(x)
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        x = self.dropout(x)
        x = x.view(-1)
        return x

def train_model(model, train_loader, criterion, optimizer, scheduler, epochs):
    clip_value = 1.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            labels = labels.view(-1)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")

def accuracy(predictions, targets, threshold=0.1):
    absolute_errors = torch.abs(predictions - targets)
    correct_predictions = (absolute_errors <= threshold).float()
    accuracy = correct_predictions.mean().item()
    return accuracy

def evaluate_regression_model(model, test_loader, criterion, threshold=0.1):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    total_accuracy = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            labels = labels.view(-1)
            loss = criterion(outputs.squeeze(), labels)
            total_loss += loss.item()
            
            # 정확도 계산
            acc = accuracy(outputs, labels, threshold=threshold)
            total_accuracy += acc

            total_samples += labels.size(0)

    mean_loss = total_loss / len(test_loader)
    mean_accuracy = total_accuracy / len(test_loader)
    print(f"Test Loss: {mean_loss}, Accuracy: {mean_accuracy}")

    return mean_loss, mean_accuracy

In [ ]:

# 데이터를 그래프로 변환하는 함수
def create_graph_data(df, features, target):
    # 데이터 정규화
    scaler = StandardScaler()
    df[features] = scaler.fit_transform(df[features])

    # 그래프 구조 생성
    adjacency_matrix = torch.tensor(df.corr().abs().values, dtype=torch.float32)

    # 데이터와 인접 행렬 반환
    x = torch.tensor(df[features].values, dtype=torch.float32)
    y = torch.tensor(df[target].values, dtype=torch.float32)
    return x, adjacency_matrix, y

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc1 = nn.Linear(input_dim, hidden_dim)
        self.gc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, adjacency_matrix):
        x = torch.matmul(x, self.gc1.weight.t())  # 가중치 행렬을 전치하여 곱셈
        adjacency_matrix = adjacency_matrix[:64, :64]  # 일부 노드만 사용하도록 조정
        x = torch.matmul(adjacency_matrix, x.t()).t()  # x를 변환하여 행렬 곱셈이 가능하게 함
        x = torch.relu(x)
        x = torch.matmul(adjacency_matrix, x.t()).t()  # x를 다시 변환하여 행렬 곱셈이 가능하게 함
        x = self.gc2(x)
        return x
    
train_x, train_adjacency, train_y = create_graph_data(trainDF, features, target)
test_x, test_adjacency, test_y = create_graph_data(testDF, features, target)

input_dim = len(features)
hidden_dim = 64
output_dim = 1  # 예측할 값이 하나이므로 output_dim을 1로 설정
model = GCN(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(train_x, train_adjacency)
    loss = criterion(output, train_y)
    loss.backward()
    optimizer.step()

# 테스트
with torch.no_grad():
    test_output = model(test_x, test_adjacency)
    test_loss = criterion(test_output, test_y)
    print(f'Test Loss: {test_loss.item()}')
    
evaluate_regression_model(model, testDF, criterion, threshold=0.1)

In [ ]:
from torch.utils.data import TensorDataset
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_losses = []
fold_accuracies = []

for fold, (train_index, test_index) in enumerate(kf.split(train_x_tensor)):
    print(f"Fold {fold + 1}")
    
    fold_train_x, fold_test_x = train_x_tensor[train_index], train_x_tensor[test_index]
    fold_train_y, fold_test_y = train_y_tensor[train_index] / 60, train_y_tensor[test_index] / 60

    # Move fold data to PyTorch tensors and then to the device
    fold_train_x = fold_train_x.to(device)
    fold_train_y = fold_train_y.to(device)
    fold_test_x = fold_test_x.to(device)
    fold_test_y = fold_test_y.to(device)

    # Create DataLoader for training and testing data
    train_dataset = TensorDataset(fold_train_x, fold_train_y)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(fold_test_x, fold_test_y)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    # Specify the number of time steps and features
    timesteps = 10
    feature_num = len(features)

    # Create an instance of the AirlineDelayPredictionModel class
    model = AirlineDelayPredictionModel(timesteps, feature_num).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.2)
    # Train the model
    train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20)
    
    # Evaluate the model on the test data
    fold_loss, fold_accuracy = evaluate_regression_model(model, test_loader, criterion, threshold=0.1)
    print(f"Fold {fold + 1} - Test Loss: {fold_loss}, Accuracy: {fold_accuracy}")

    fold_losses.append(fold_loss)
    fold_accuracies.append(fold_accuracy)

average_loss = sum(fold_losses) / len(fold_losses)
average_accuracy = sum(fold_accuracies) / len(fold_accuracies)

print(f"Overall Performance - Average Test Loss: {average_loss}, Average Accuracy: {average_accuracy}")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

scaler = MinMaxScaler()

# 학습 데이터 스케일링
train_x_scaled = scaler.fit_transform(train_x)

# PyTorch 텐서로 변환
train_x_tensor = torch.tensor(train_x_scaled, dtype=torch.float32)
train_y_tensor = torch.tensor(train_y.values, dtype=torch.float32)

# 테스트 데이터 스케일링
test_x_scaled = scaler.fit_transform(test_x)

# PyTorch 텐서로 변환
test_x_tensor = torch.tensor(test_x_scaled, dtype=torch.float32)
test_y_tensor = torch.tensor(test_y.values, dtype=torch.float32)

#test_x_scaled = scaler.transform(test_x.values)

# Convert data to PyTorch tensors
#train_x_tensor, train_y_tensor = torch.tensor(train_x_scaled, dtype=torch.float32), torch.tensor(train_y, dtype=torch.float32)
#test_x_tensor, test_y_tensor = torch.tensor(test_x_scaled, dtype=torch.float32), torch.tensor(test_y.values, dtype=torch.float32)



In [ ]:
final_test_dataset = TensorDataset(test_x, test_y)
final_test_loader = DataLoader(final_test_dataset, batch_size=32, shuffle=False)

# Create a new instance of the model for the final test
final_model = AirlineDelayPredictionModel(timesteps, features)

# Train the final model on the entire training set
final_optimizer = optim.Adam(final_model.parameters(), lr=0.001)
final_train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
train_model(final_model, final_train_loader, criterion, final_optimizer, epochs=10)

# Evaluate the final model on the entire test set
final_test_loss, final_test_accuracy = evaluate_model(final_model, final_test_loader, criterion)
print(f"Final Test Loss: {final_test_loss}, Final Accuracy: {final_test_accuracy}")